# Match to Known Molecules
The standard approach to assessing whether an inorganic material is synthesizable is to see if someone has made it and reported it in the literature. 
We emulate this process for inorganic molecules by seeing if each molecule is found in PubChem or the EMO dataset.

In [ ]:
%matplotlib inline 
from matplotlib import pyplot as plt
from multiprocessing import Pool
from functools import partial
from pathlib import Path
from typing import List
from rdkit import Chem
from tqdm import tqdm
import pandas as pd
import numpy as np
import gzip
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

## Define location of experimental databases for cross referencing

In [ ]:
known_molecules = ['data/raw/PCH.csv.gz', 'data/raw/EMO.csv.gz']
chunk_size = 1e6

## Load in the QM9 dataset
It is a small dataset, so we're going to read it into memory

In [ ]:
qm9_data = pd.read_csv('data/clean/qm9.csv')

## Make a lookup function
We want to return if a QM9 molecule is found in a certain set of molecules. It will take in a list of SMILES strings (the format provided by NVBL) and return a list of matching indices

In [ ]:
def match(smiles, qm9_set, qm9_mols):
    """Get the index of a molecule in QM9
    
    Args:
        mol - Molecule to match
        qm9_set - Set of all molecules in QM9. Used to check for membership
        qm9_mols - List of molecules in QM9. Used to get index
    Returns:
        Index of molecule, if exists. None otherwise
    """
    # Convert the SMILES to InChI
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        inchi = Chem.MolToInchi(mol)
    except:
        return None
    
    # Check if it is in the list
    if inchi in qm9_set:
        # A hit is rare, so our unoptimized "index" function is OK
        return qm9_mols.index(inchi)

In [ ]:
def lookup_molecules(qm9_mols: List[str], chunk_mols: List[str]) -> List[int]:
    """Find if any QM9 molecules are contained in this chunk of molecules
    
    Args:
        qm9_mols: List of InChI strings of the molecules from the QM9
        chunk_mols: List of _SMILES_ strings of the molecules contained within QM9
    Returns:
        List of molecules that were matched
    """
    
    # Get a set for faster lookup
    qm9_set = set(qm9_mols)
    
    # Make a lookup function 
    fun = partial(match, qm9_set=qm9_set, qm9_mols=qm9_mols)
    with Pool() as p:
        output = set(x for x in p.map(fun, chunk_mols) if x is not None)
        
    return list(output)

## Find QM9 molecules in the larger set
Do it in chunks of $10^6$ molecules, which should fit in memory

In [ ]:
qm9_mols = qm9_data['inchi_0'].tolist()

In [ ]:
for path in known_molecules:    
    # Get a name for the dataset
    name = Path(path).name[:3]

    # Initialize the list of molecules found and chunk
    chunk = []
    found = set()
    with gzip.open(path, 'rt') as fp:
        for line in tqdm(fp, desc=name):
            # Get the SMILES string from that line
            try:
                _, _, smiles = line.strip().split(",", 3)
            except:
                continue 
            chunk.append(smiles)

            if len(chunk) > chunk_size:
                # Run the computation
                found_chunk = lookup_molecules(qm9_mols, chunk)
                found.update(found_chunk)
                chunk.clear()

        # Run the calculation on the last chunk
        found.update(lookup_molecules(qm9_mols, chunk))
        
    # Store them in a row of the dataframe
    qm9_data[f'in_{name}'] = False
    if len(found) > 0:
        qm9_data.loc[list(found), f'in_{name}'] = True

Print out what fraction of QM9 we matched

In [ ]:
qm9_data.iloc[:, -len(known_molecules):].mean()

## add reported flag

In [ ]:
reported = ((qm9_data['in_PCH']) | (qm9_data['in_EMO'])) # in pubchem (PCH) or EMO

qm9_data['Reported'] = reported

## Save results to disk
Overwrite the clean copy of the data

In [ ]:
qm9_data.to_csv('data/clean/qm9.csv', index=False)